In [49]:
from docx import Document
from docx.shared import Pt
from docx.enum.table import WD_ALIGN_VERTICAL
import os

# Criar documento
doc = Document()

# Adicionar título
titulo = doc.add_heading('Comparação entre Grupos Sociais', level=1)
titulo.alignment = 1  # Centralizado

# Adicionar tabela
tabela = doc.add_table(rows=1, cols=4)
tabela.style = 'Light Shading'

# Cabeçalhos
cabecalhos = tabela.rows[0].cells
cabecalhos[0].text = 'Variável'
cabecalhos[1].text = 'Vulnerável 🟥\n(Média: 500.03)'
cabecalhos[2].text = 'Intermediário 🟨\n(Média: 575.62)'
cabecalhos[3].text = 'Privilegiado 🟩\n(Média: 660.00)'

# Dados
dados = [
    ['Escolaridade do Pai (Q001)', '2.64', '3.92', '5.16'],
    ['Escolaridade da Mãe (Q002)', '2.98', '4.29', '5.34'],
    ['Ocupação do Pai (Q003)', '1.59', '2.79', '3.50'],
    ['Ocupação da Mãe (Q004)', '1.31', '2.63', '3.42'],
    ['Renda Familiar (Q006)', '1.55', '3.56', '10.82'],
    ['Computador em Casa (Q024)', '0.19', '0.71', '2.21'],
    ['Internet em Casa (Q025)', '0.83', '0.99', '1.00'],
    ['Escola Privada (TP_ESCOLA)', '0.03', '0.47', '0.94'],
    ['Cor/Raça (TP_COR_RACA)', '0.19', '0.38', '0.70']
]

# Adicionar linhas
for linha in dados:
    celulas = tabela.add_row().cells
    for i, valor in enumerate(linha):
        celulas[i].text = str(valor)
        celulas[i].vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# Ajustar estilo
for row in tabela.rows:
    for cell in row.cells:
        paragraphs = cell.paragraphs
        for paragraph in paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(10)

# Definir o caminho completo para salvar
caminho_completo = r'C:\Users\CWS\Documents\TCC\Meu tcc\clusters\comparacao_grupos_sociais.docx'

# Criar diretório se não existir
os.makedirs(os.path.dirname(caminho_completo), exist_ok=True)

# Salvar documento
doc.save(caminho_completo)
print(f"Documento Word gerado com sucesso em: {caminho_completo}")

Documento Word gerado com sucesso em: C:\Users\CWS\Documents\TCC\Meu tcc\clusters\comparacao_grupos_sociais.docx


In [43]:
!pip install python-docx

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score , calinski_harabasz_score, pairwise_distances
import plotly.express as px
from geobr import read_state
from matplotlib.colors import ListedColormap
import unicodedata

# Configurações iniciais
plt.style.use('ggplot')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)

# -----------------------------
# ETAPA 1: Carregar e preparar os dados
# -----------------------------
# Função para remover acentos e padronizar
def normalizar_texto(texto):
    if pd.isnull(texto):
        return texto
    texto = str(texto).strip().title()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Dicionário: capitais do Nordeste → siglas dos estados
capitais_para_uf = {
    'São Luís': 'MA',
    'Teresina': 'PI',
    'Fortaleza': 'CE',
    'Natal': 'RN',
    'João Pessoa': 'PB',
    'Recife': 'PE',
    'Maceió': 'AL',
    'Aracaju': 'SE',
    'Salvador': 'BA'
}

# Ajustar o dicionário também para ter nomes sem acento
capitais_para_uf_normalizado = {normalizar_texto(k): v for k, v in capitais_para_uf.items()}

# Carregar dados do ENEM (ajuste o caminho)
caminho_arquivo = r"C:\Users\CWS\Documents\meu\Bases clustering\microdados_enem_tratados_5.csv"
df = pd.read_csv(caminho_arquivo, sep=";", encoding="utf-8")

df['NO_MUNICIPIO_PROVA_NORMALIZADO'] = df['NO_MUNICIPIO_ESC'].apply(normalizar_texto)

# Mapeia sigla da UF a partir da capital normalizada
capitais_para_uf_normalizado = {normalizar_texto(k): v for k, v in capitais_para_uf.items()}
df['SG_UF_ESC'] = df['NO_MUNICIPIO_PROVA_NORMALIZADO'].map(capitais_para_uf_normalizado)

faltantes = df[df['SG_UF_ESC'].isnull()]['NO_MUNICIPIO_ESC'].unique()
print("Capitais não mapeadas:", faltantes)

# Filtrar apenas alunos do Nordeste
ufs_nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA']
df_nordeste = df[df['SG_UF_ESC'].isin(ufs_nordeste)].copy()

# Calcular nota média
df_nordeste['NU_NOTA_MEDIA'] = df_nordeste[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)

# Selecionar variáveis para clusterização
variaveis = [
    'Q001',  # Escolaridade do pai
    'Q002',  # Escolaridade da mãe
    'Q003',  # Ocupação do pai
    'Q004',  # Ocupação da mãe
    'Q006',  # Renda familiar
    'Q024',  # Computador em casa
    'Q025',  # Internet em casa
    'TP_ESCOLA',  # Tipo de escola
    'TP_COR_RACA',  # Raça/cor
    'NU_NOTA_MEDIA'  # Desempenho acadêmico
]

df_cluster = df_nordeste[variaveis].dropna().copy()

# -----------------------------
# ETAPA 2: Pré-processamento
# -----------------------------

# Codificar variáveis categóricas
encoder = LabelEncoder()
df_cluster['TP_ESCOLA'] = encoder.fit_transform(df_cluster['TP_ESCOLA'])
df_cluster['TP_COR_RACA'] = encoder.fit_transform(df_cluster['TP_COR_RACA'])


# Normalizar os dados
scaler = StandardScaler()
X = scaler.fit_transform(df_cluster)

# -----------------------------
# ETAPA 3: Determinar número ótimo de clusters

range_clusters = range(1, 11)  # Definir o range de clusters a testar
# -----------------------------

# Método do cotovelo (melhorado)
wcss = []
for i in range_clusters:
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Identificar o "cotovelo" programaticamente
elbow_diff = np.diff(wcss)
elbow_angle = np.diff(elbow_diff)
optimal_elbow = np.argmin(elbow_angle) + 2  # +2 porque diff reduz em 2 o tamanho

plt.figure(figsize=(12, 6))
plt.plot(range_clusters, wcss, marker='o', label='WCSS')
plt.axvline(x=optimal_elbow, color='r', linestyle='--', 
            label=f'Cotovelo sugerido (k={optimal_elbow})')
plt.title('Método do Cotovelo - Número Ótimo de Clusters', fontsize=14)
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS (Inércia)')
plt.legend()
plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\elbow_method_improved.png", dpi=300)
plt.close()

print(f"\nMétodo do Cotovelo sugere: {optimal_elbow} clusters")

# Método da silhueta (melhorado)
silhouette_scores = []
range_silhouette = range(2, 11)  # Silhueta requer pelo menos 2 clusters

for i in range_silhouette:
    kmeans = KMeans(n_clusters=i, random_state=42)
    labels = kmeans.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, labels))

optimal_silhouette = range_silhouette[np.argmax(silhouette_scores)]

plt.figure(figsize=(12, 6))
plt.plot(range_silhouette, silhouette_scores, marker='o', label='Score de Silhueta')
plt.axvline(x=optimal_silhouette, color='g', linestyle='--', 
            label=f'Ótimo sugerido (k={optimal_silhouette})')
plt.title('Método da Silhueta - Número Ótimo de Clusters', fontsize=14)
plt.xlabel('Número de Clusters')
plt.ylabel('Score de Silhueta')
plt.legend()
plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\silhouette_method_improved.png", dpi=300)
plt.close()

print(f"Método da Silhueta sugere: {optimal_silhouette} clusters")

# Métrica Davies-Bouldin
from sklearn.metrics import davies_bouldin_score

db_scores = []
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    labels = kmeans.fit_predict(X)
    db_scores.append(davies_bouldin_score(X, labels))

optimal_db = range(2, 11)[np.argmin(db_scores)]

plt.figure(figsize=(12, 6))
plt.plot(range(2, 11), db_scores, marker='o')
plt.axvline(x=optimal_db, color='b', linestyle='--',
            label=f'Ótimo sugerido (k={optimal_db})')

plt.title('Método Davies-Bouldin - Número Ótimo de Clusters', fontsize=18)
plt.xlabel('Número de Clusters', fontsize=16)
plt.ylabel('Davies-Bouldin Score (menor é melhor)', fontsize=16)
plt.legend(fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\davies_bouldin_method.png",
            dpi=300, bbox_inches="tight")
plt.close()

print(f"Método Davies-Bouldin sugere: {optimal_db} clusters")

# Opção 1: Usar o valor da maioria
# Definir 2 clusters conforme Silhueta e Davies-Bouldin
n_clusters = 2

# Aplicar K-means
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_cluster['cluster'] = kmeans.fit_predict(X)

# Adicionar informações geográficas de volta
df_cluster['SG_UF_ESC'] = df_nordeste.loc[df_cluster.index, 'SG_UF_ESC']

# Testar também com 3 clusters para comparação
kmeans_k3 = KMeans(n_clusters=3, random_state=42)
labels_k3 = kmeans_k3.fit_predict(X)
centroids_k3 = kmeans_k3.cluster_centers_

# Métricas para k=2 (já calculado)
silh_k2 = silhouette_score(X, df_cluster['cluster']) 
# Métricas para k=3
silh_k3 = silhouette_score(X, labels_k3)

print(f"\nComparação de métodos:")
print(f"Silhueta k=2: {silh_k2:.3f} | Silhueta k=3: {silh_k3:.3f}")
print(f"Relação de qualidade: {(silh_k3/silh_k2)*100:.1f}%")

# Calcular R² para k=3
inter_cluster_dist_k3 = pairwise_distances(centroids_k3)
intra_cluster_dist_k3 = [np.mean(pairwise_distances(X[labels_k3==i])) for i in range(3)]
r_squared_k3 = (inter_cluster_dist_k3.mean() - np.mean(intra_cluster_dist_k3)) / inter_cluster_dist_k3.mean()

# Calcular R² para k=2
inter_cluster_dist_k2 = pairwise_distances(kmeans.cluster_centers_)
intra_cluster_dist_k2 = [np.mean(pairwise_distances(X[df_cluster['cluster']==i])) for i in range(2)]
r_squared_k2 = (inter_cluster_dist_k2.mean() - np.mean(intra_cluster_dist_k2)) / inter_cluster_dist_k2.mean()

print(f"R² entre clusters k=2: {r_squared_k2:.3f} | k=3: {r_squared_k3:.3f}")
print(f"Ganho explicativo: {(r_squared_k3/r_squared_k2-1)*100:.1f}%")


# ETAPA 4: Pós-clusterização (após kmeans.fit_predict)
# ---------------------------------------------------

# Renomear os clusters para análise interpretável
cluster_names_2 = {
    0: 'Vulnerável',
    1: 'Privilegiado'
}

# Criar nova coluna com os nomes dos clusters
df_cluster['cluster_nome'] = df_cluster['cluster'].map(cluster_names)

# Atribuir nomes legíveis
df_cluster['cluster_nome'] = df_cluster['cluster'].map(cluster_names_2)
df_cluster['cluster'] = pd.Categorical(
    df_cluster['cluster_nome'],
    categories=['Vulnerável', 'Privilegiado'],
    ordered=True
)

# -----------------------------
# ETAPA 5: Visualização dos Clusters
# -----------------------------

# PCA para variância explicada acumulada
pca_temp = PCA().fit(X)
explained = np.cumsum(pca_temp.explained_variance_ratio_)

# Determinar quantos componentes atingem 80% de variância
n_componentes_80 = np.argmax(explained >= 0.80) + 1
print(f"Componentes para 80% da variância: {n_componentes_80}")


# Aplicar PCA com esse número
pca = PCA(n_components=n_componentes_80)
X_pca = pca.fit_transform(X)

# Gráfico 3D interativo
fig = px.scatter_3d(
    x=X_pca[:, 0], y=X_pca[:, 1], z=X_pca[:, 2],  # Agora todas as 3 dimensões existem
    color=df_cluster['cluster'],
    labels={'color': 'Cluster'},
    title='Visualização 3D dos Clusters (2 grupos)'
)
fig.write_html(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\3d_clusters_2.html")

# Gráfico de dispersão 2D
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x=X_pca[:, 0], y=X_pca[:, 1],
    hue=df_cluster['cluster'],
    palette='Set2', s=60, alpha=0.7
)

plt.title('Visualização dos Clusters (PCA)', fontsize=20)
plt.xlabel('Componente Principal 1', fontsize=18)
plt.ylabel('Componente Principal 2', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(title='Cluster', fontsize=16, title_fontsize=18)

plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\pca_clusters_2.png",
            dpi=300, bbox_inches="tight")
plt.close()

# Variância explicada
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print("\nVariância explicada pelas componentes PCA:")
print(f"PC1: {explained_variance[0]:.2%}")
print(f"PC2: {explained_variance[1]:.2%} (Acumulada: {cumulative_variance[1]:.2%})")
print(f"PC3: {explained_variance[2]:.2%} (Acumulada: {cumulative_variance[2]:.2%})")

# Plot variância explicada
componentes = len(explained_variance)
plt.figure(figsize=(10, 6))
plt.bar(range(1, componentes + 1), explained_variance, alpha=0.6, label='Individual')
plt.plot(range(1, componentes + 1), cumulative_variance, marker='o', label='Acumulada')
plt.xlabel('Componentes Principais')
plt.ylabel('Variância Explicada')
plt.title('Variância Explicada pelas Componentes PCA', fontsize=14)
plt.legend()
plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\pca_variance.png", dpi=300)
plt.close()


# -----------------------------
# ETAPA 6: Análise dos Clusters
# -----------------------------

# Caracterização dos clusters
cluster_stats = df_cluster.groupby('cluster').agg({
    'Q001': 'mean',            # Escolaridade do pai
    'Q002': 'mean',            # Escolaridade da mãe
    'Q003': 'mean',            # Ocupação do pai
    'Q004': 'mean',            # Ocupação da mãe
    'Q006': 'mean',            # Renda familiar
    'Q024': 'mean',            # Computador em casa
    'Q025': 'mean',            # Internet em casa
    'TP_ESCOLA': 'mean',       # Tipo de escola
    'TP_COR_RACA': 'mean',     # Raça/cor
    'NU_NOTA_MEDIA': 'mean'    # Desempenho acadêmico
})
print("\nEstatísticas por Cluster (2  clusters):")
print(cluster_stats.round(2))

# Salvar estatísticas com separador de vírgula e ponto decimal
cluster_stats.to_csv(
    r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\cluster_stats_2.csv",
    sep=',',
    decimal='.',
    index=True
)


# -----------------------------
# ETAPA 7: Visualização por Estado
# -----------------------------

# -----------------------------
# ETAPA 7: Visualização por Estado - CÓDIGO CORRIGIDO
# -----------------------------

# 1. Gráfico de barras (distribuição proporcional por estado)
cluster_por_estado = pd.crosstab(df_cluster['SG_UF_ESC'], df_cluster['cluster_nome'], normalize='index')

plt.figure(figsize=(12, 8))
cluster_por_estado[['Vulnerável', 'Privilegiado']].plot(
    kind='bar', 
    stacked=True, 
    color=['#d73027', '#fee08b', '#1a9850']
)
plt.title('Distribuição Proporcional dos Clusters por Estado', fontsize=14)
plt.xlabel('Estado')
plt.ylabel('Proporção')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\clusters_por_estado_2.png", dpi=300)
plt.close()

# 2. Mapa coroplético (cluster predominante por estado)
estados = read_state(year=2020)

# Verificar nome da coluna de siglas
print("\nColunas disponíveis no geodataframe:", estados.columns.tolist())

# O nome correto da coluna pode variar - ajuste conforme necessário
coluna_sigla = 'abbrev_state'  # ou 'sigla' dependendo da versão do geobr

estados_nordeste = estados[estados[coluna_sigla].isin(ufs_nordeste)].copy()


# Calcular cluster predominante
cluster_predominante = df_cluster.groupby('SG_UF_ESC')['cluster_nome'].agg(
    lambda x: x.value_counts().index[0]
).reset_index()

# Juntar com o mapa
estados_nordeste = estados_nordeste.merge(
    cluster_predominante.rename(columns={'SG_UF_ESC': coluna_sigla}),
    on=coluna_sigla,
    how='left'
)

# Definir ordem e cores
# Cores e ordem
ordem_clusters_2 = ['Vulnerável', 'Privilegiado']
cores_clusters_2 = ['#d73027', '#1a9850']
cmap_2 = ListedColormap(cores_clusters_2)

estados_nordeste['cluster'] = pd.Categorical(
    estados_nordeste['cluster_nome'],
    categories=ordem_clusters_2,
    ordered=True
)

# Plotar o mapa
fig, ax = plt.subplots(figsize=(14, 10))
estados_nordeste.plot(
    column='cluster',
    categorical=True,
    cmap=cmap_clusters,
    edgecolor='black',
    linewidth=0.8,
    legend=True,
    ax=ax,
    missing_kwds={'color': 'lightgrey'}
)

# Ajustar legenda
legenda = ax.get_legend()
if legenda:
    legenda.set_title('Perfil Socioeconômico')
    for texto, rotulo in zip(legenda.get_texts(), ordem_clusters):
        texto.set_text(rotulo)

# Adicionar rótulos
for idx, row in estados_nordeste.iterrows():
    centroid = row.geometry.centroid
    ax.annotate(
        text=row[coluna_sigla],
        xy=(centroid.x, centroid.y),
        ha='center',
        va='center',
        fontsize=10,
        bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='none', alpha=0.8)
    )

ax.set_title('Perfil Socioeconômico Predominante por Estado no Nordeste', fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\mapa_clusters_final_2.png", dpi=300, bbox_inches='tight')
plt.close()

Capitais não mapeadas: []

Método do Cotovelo sugere: 5 clusters
Método da Silhueta sugere: 2 clusters
Método Davies-Bouldin sugere: 2 clusters

Comparação de métodos:
Silhueta k=2: 0.322 | Silhueta k=3: 0.216
Relação de qualidade: 67.0%
R² entre clusters k=2: -0.712 | k=3: -0.437
Ganho explicativo: -38.6%


NameError: name 'cluster_names' is not defined

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import plotly.express as px
from geobr import read_state
from matplotlib.colors import ListedColormap
import unicodedata
from scipy.stats import zscore

# Configurações iniciais
plt.style.use('ggplot')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)

# -----------------------------
# ETAPA 1: Carregar e preparar os dados
# -----------------------------
# Função para remover acentos e padronizar
def normalizar_texto(texto):
    if pd.isnull(texto):
        return texto
    texto = str(texto).strip().title()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Dicionário: capitais do Nordeste → siglas dos estados
capitais_para_uf = {
    'São Luís': 'MA',
    'Teresina': 'PI',
    'Fortaleza': 'CE',
    'Natal': 'RN',
    'João Pessoa': 'PB',
    'Recife': 'PE',
    'Maceió': 'AL',
    'Aracaju': 'SE',
    'Salvador': 'BA'
}

# Ajustar o dicionário também para ter nomes sem acento
capitais_para_uf_normalizado = {normalizar_texto(k): v for k, v in capitais_para_uf.items()}

# Carregar dados do ENEM (ajuste o caminho)
caminho_arquivo = r"C:\Users\CWS\Documents\meu\Meu tcc\microdados_enem_tratados2.csv"
df = pd.read_csv(caminho_arquivo, sep=";", encoding="utf-8")

df['NO_MUNICIPIO_PROVA_NORMALIZADO'] = df['NO_MUNICIPIO_PROVA'].apply(normalizar_texto)

# Mapeia sigla da UF a partir da capital normalizada
capitais_para_uf_normalizado = {normalizar_texto(k): v for k, v in capitais_para_uf.items()}
df['SG_UF_ESC'] = df['NO_MUNICIPIO_PROVA_NORMALIZADO'].map(capitais_para_uf_normalizado)

faltantes = df[df['SG_UF_ESC'].isnull()]['NO_MUNICIPIO_PROVA'].unique()
print("Capitais não mapeadas:", faltantes)

# Filtrar apenas alunos do Nordeste
ufs_nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA']
df_nordeste = df[df['SG_UF_ESC'].isin(ufs_nordeste)].copy()

# Calcular nota média
df_nordeste['NU_NOTA_MEDIA'] = df_nordeste[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)

# Selecionar variáveis para clusterização
variaveis = [
    'Q001',  # Escolaridade do pai
    'Q002',  # Escolaridade da mãe
    'Q003',  # Ocupação do pai
    'Q004',  # Ocupação da mãe
    'Q006',  # Renda familiar
    'Q024',  # Computador em casa
    'Q025',  # Internet em casa
    'TP_ESCOLA',  # Tipo de escola
    'TP_COR_RACA',  # Raça/cor
    'NU_NOTA_MEDIA',  # Desempenho acadêmico
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO' 
]

df_cluster = df_nordeste[variaveis].dropna().copy()

# -----------------------------
# ETAPA 2: Pré-processamento
# -----------------------------

# Codificar variáveis categóricas
encoder = LabelEncoder()
df_cluster['TP_ESCOLA'] = encoder.fit_transform(df_cluster['TP_ESCOLA'])
df_cluster['TP_COR_RACA'] = encoder.fit_transform(df_cluster['TP_COR_RACA'])

# Selecionar colunas numéricas para clusterização
X = df_cluster.select_dtypes(include=[np.number]).values


# Verificar porcentagem de outliers para diferentes limiares
for limite in [2, 2.5, 3, 3.5]:
    outliers = (np.abs(zscore(X)) > limite).any(axis=1)
    porcentagem = 100 * outliers.sum() / len(X)
    print(f"Limiar {limite}: {porcentagem:.2f}% das amostras seriam removidas.")
    

# Definir limiar final e remover outliers
limite_zscore = 3
z_scores = np.abs(zscore(X))
filtro_sem_outliers = (z_scores < limite_zscore).all(axis=1)
X_sem_outliers = X[filtro_sem_outliers]
df_cluster_sem_outliers = df_cluster[filtro_sem_outliers].copy()


# Normalizar os dados após remoção de outliers
scaler = StandardScaler()
X = scaler.fit_transform(df_cluster_sem_outliers)
# -----------------------------
# ETAPA 3: Determinar número ótimo de clusters
# -----------------------------

range_clusters = range(1, 11)  # Definir o range de clusters a testar
# -----------------------------

# Método do cotovelo (melhorado)
wcss = []
for i in range_clusters:
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

# Identificar o "cotovelo" programaticamente
elbow_diff = np.diff(wcss)
elbow_angle = np.diff(elbow_diff)
optimal_elbow = np.argmin(elbow_angle) + 2  # +2 porque diff reduz em 2 o tamanho

plt.figure(figsize=(12, 6))
plt.plot(range_clusters, wcss, marker='o', label='WCSS')
plt.axvline(x=optimal_elbow, color='r', linestyle='--', 
            label=f'Cotovelo sugerido (k={optimal_elbow})')
plt.title('Método do Cotovelo - Número Ótimo de Clusters', fontsize=14)
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS (Inércia)')
plt.legend()
plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\elbow_method_improved.png", dpi=300)
plt.close()

print(f"\nMétodo do Cotovelo sugere: {optimal_elbow} clusters")

# Método da silhueta (melhorado)
silhouette_scores = []
range_silhouette = range(2, 11)  # Silhueta requer pelo menos 2 clusters

for i in range_silhouette:
    kmeans = KMeans(n_clusters=i, random_state=42)
    labels = kmeans.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, labels))

optimal_silhouette = range_silhouette[np.argmax(silhouette_scores)]

plt.figure(figsize=(12, 6))
plt.plot(range_silhouette, silhouette_scores, marker='o', label='Score de Silhueta')
plt.axvline(x=optimal_silhouette, color='g', linestyle='--', 
            label=f'Ótimo sugerido (k={optimal_silhouette})')
plt.title('Método da Silhueta - Número Ótimo de Clusters', fontsize=14)
plt.xlabel('Número de Clusters')
plt.ylabel('Score de Silhueta')
plt.legend()
plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\silhouette_method_improved.png", dpi=300)
plt.close()

print(f"Método da Silhueta sugere: {optimal_silhouette} clusters")

# Métrica Davies-Bouldin
from sklearn.metrics import davies_bouldin_score

db_scores = []
for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    labels = kmeans.fit_predict(X)
    db_scores.append(davies_bouldin_score(X, labels))

optimal_db = range(2, 11)[np.argmin(db_scores)]

plt.figure(figsize=(12, 6))
plt.plot(range(2, 11), db_scores, marker='o', label='Davies-Bouldin Score')
plt.axvline(x=optimal_db, color='b', linestyle='--', 
            label=f'Ótimo sugerido (k={optimal_db})')
plt.title('Método Davies-Bouldin - Número Ótimo de Clusters', fontsize=14)
plt.xlabel('Número de Clusters')
plt.ylabel('Davies-Bouldin Score (menor é melhor)')
plt.legend()
plt.grid(True)
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\davies_bouldin_method.png", dpi=300)
plt.close()

print(f"Método Davies-Bouldin sugere: {optimal_db} clusters")


# Definir número de clusters (ajuste conforme análise acima)
n_clusters = 3

# Aplicar K-means apenas aos dados sem outliers
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X)  # X já está sem outliers

# Criar nova coluna APENAS nas linhas sem outliers
df_cluster_sem_outliers['cluster'] = clusters  # Use o dataframe filtrado

# Adicionar informações geográficas corretamente
df_cluster_sem_outliers['SG_UF_ESC'] = df_nordeste.loc[df_cluster_sem_outliers.index, 'SG_UF_ESC']

# ETAPA 4: Pós-clusterização (após kmeans.fit_predict)
# ---------------------------------------------------

# Renomear os clusters para análise interpretável
cluster_names = {
    0: 'Intermediário',
    1: 'Vulnerável', 
    2: 'Privilegiado'
}

# Aplicar nos dados SEM OUTLIERS
df_cluster_sem_outliers['cluster_nome'] = df_cluster_sem_outliers['cluster'].map(cluster_names)

# Converter para categoria ordenada
df_cluster_sem_outliers['cluster'] = pd.Categorical(
    df_cluster_sem_outliers['cluster_nome'],
    categories=['Vulnerável', 'Intermediário', 'Privilegiado'],
    ordered=True
)
# -----------------------------
# ETAPA 5: Visualização dos Clusters
# -----------------------------

# Redução para 3 componentes
pca = PCA(n_components=3)  # Alterado para 3
X_pca = pca.fit_transform(X)

# Gráfico 3D interativo
fig = px.scatter_3d(
    x=X_pca[:, 0], y=X_pca[:, 1], z=X_pca[:, 2],  # Agora todas as 3 dimensões existem
    color=df_cluster_sem_outliers['cluster'],
    labels={'color': 'Cluster'},
    title='Visualização 3D dos Clusters'
)
fig.write_html(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\3d_clusters.html")

# Gráfico de dispersão 2D
plt.figure(figsize=(12, 8))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue= df_cluster_sem_outliers['cluster'], palette='Set2', s=50, alpha=0.7)
plt.title('Visualização dos Clusters (PCA)', fontsize=14)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\pca_clusters.png", dpi=300)
plt.close()

# ETAPA 6: Análise dos Clusters
# -----------------------------

# Caracterização dos clusters

cluster_stats = df_cluster_sem_outliers.groupby('cluster_nome').agg({
    'Q001': 'mean',            # Escolaridade do pai
    'Q002': 'mean',            # Escolaridade da mãe
    'Q003': 'mean',            # Ocupação do pai
    'Q004': 'mean',            # Ocupação da mãe
    'Q006': 'mean',            # Renda familiar
    'Q024': 'mean',            # Computador em casa
    'Q025': 'mean',            # Internet em casa
    'TP_ESCOLA': 'mean',       # Tipo de escola
    'TP_COR_RACA': 'mean',     # Raça/cor
    'NU_NOTA_MEDIA': 'mean'    # Desempenho acadêmico
})
print("\nEstatísticas por Cluster (3 clusters):")
print(cluster_stats.round(2))

# Salvar estatísticas em formato compatível com Excel (PT-BR)
cluster_stats.to_csv(
    r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\cluster_stats_3.csv",
    sep=';',        # separador de colunas
    decimal=',',    # separador decimal
    index=True
)
# -----------------------------
# ETAPA 7: Visualização por Estado (3 clusters)
# -----------------------------

# 1. Gráfico de barras (distribuição proporcional por estado)
cluster_por_estado = pd.crosstab(df_cluster_sem_outliers['SG_UF_ESC'], 
                                df_cluster_sem_outliers['cluster_nome'], 
                                normalize='index')

plt.figure(figsize=(12, 8))
cluster_por_estado[['Vulnerável', 'Intermediário', 'Privilegiado']].plot(
    kind='bar', 
    stacked=True, 
    color=['#d73027', '#fee08b', '#1a9850']  # Cores para 3 clusters
)
plt.title('Distribuição Proporcional dos Clusters por Estado', fontsize=14)
plt.xlabel('Estado')
plt.ylabel('Proporção')
plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\clusters_por_estado_3.png", dpi=300)
plt.close()

# 2. Mapa coroplético (cluster predominante por estado)
estados = read_state(year=2020)
coluna_sigla = 'abbrev_state'  # Confirmar no seu ambiente

estados_nordeste = estados[estados[coluna_sigla].isin(ufs_nordeste)].copy()

# Calcular cluster predominante (agora com 3 categorias)
cluster_predominante = df_cluster_sem_outliers.groupby('SG_UF_ESC')['cluster_nome'].agg(
    lambda x: x.value_counts().index[0]
).reset_index()

# Juntar com o mapa
estados_nordeste = estados_nordeste.merge(
    cluster_predominante.rename(columns={'SG_UF_ESC': coluna_sigla}),
    on=coluna_sigla,
    how='left'
)

# Definir ordem e cores para 3 clusters
ordem_clusters = ['Vulnerável', 'Intermediário', 'Privilegiado']
cores_clusters = ['#d73027', '#fee08b', '#1a9850']
cmap_clusters = ListedColormap(cores_clusters)

estados_nordeste['cluster'] = pd.Categorical(
    estados_nordeste['cluster_nome'],
    categories=ordem_clusters,
    ordered=True
)

# Plotar o mapa
fig, ax = plt.subplots(figsize=(14, 10))
estados_nordeste.plot(
    column='cluster',
    categorical=True,
    cmap=cmap_clusters,
    edgecolor='black',
    linewidth=0.8,
    legend=True,
    ax=ax,
    missing_kwds={'color': 'lightgrey'}
)

# Ajustar legenda
legenda = ax.get_legend()
if legenda:
    legenda.set_title('Perfil Socioeconômico')
    for texto, rotulo in zip(legenda.get_texts(), ordem_clusters):
        texto.set_text(rotulo)

# Adicionar rótulos dos estados
for idx, row in estados_nordeste.iterrows():
    centroid = row.geometry.centroid
    ax.annotate(
        text=row[coluna_sigla],
        xy=(centroid.x, centroid.y),
        ha='center',
        va='center',
        fontsize=10,
        bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='none', alpha=0.8)
    )

ax.set_title('Perfil Socioeconômico Predominante por Estado no Nordeste', fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.savefig(r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\mapa_clusters_final_3.png", 
           dpi=300, bbox_inches='tight')
plt.close() 

# ETAPA EXTRA: Nota Média por Área do Conhecimento por Cluster
# -------------------------------------------------------------

# Selecionar apenas colunas relevantes e renomear
df_notas_cluster = df_cluster_sem_outliers[[
    'cluster_nome', 
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT', 
    'NU_NOTA_REDACAO'
]].copy()

# Renomear colunas para nomes amigáveis
df_notas_cluster.rename(columns={
    'NU_NOTA_CN': 'Ciências da Natureza',
    'NU_NOTA_CH': 'Ciências Humanas',
    'NU_NOTA_LC': 'Linguagens e Códigos',
    'NU_NOTA_MT': 'Matemática',
    'NU_NOTA_REDACAO': 'Redação'
}, inplace=True)

# Calcular médias por cluster
notas_media_por_cluster = df_notas_cluster.groupby('cluster_nome').mean().round(2)

# Ordenar pela ordem desejada dos clusters
ordem_clusters = ['Vulnerável', 'Intermediário', 'Privilegiado']
notas_media_por_cluster = notas_media_por_cluster.loc[ordem_clusters]

# Exibir resultado
print("\nMédia das Notas por Área do Conhecimento (por Cluster):")
print(notas_media_por_cluster)

# Exportar para CSV (compatível com Excel em PT-BR)
notas_media_por_cluster.to_csv(
    r"C:\Users\CWS\Documents\meu\Meu tcc\clusters\notas_media_por_cluster.csv",
    sep=';',     # separador de colunas para Excel PT-BR
    decimal=',', # separador decimal
    encoding='utf-8-sig'  # garante compatibilidade com Excel
)


Capitais não mapeadas: []
Limiar 2: 30.75% das amostras seriam removidas.
Limiar 2.5: 19.47% das amostras seriam removidas.
Limiar 3: 10.29% das amostras seriam removidas.
Limiar 3.5: 7.16% das amostras seriam removidas.

Método do Cotovelo sugere: 9 clusters
Método da Silhueta sugere: 2 clusters
Método Davies-Bouldin sugere: 2 clusters

Estatísticas por Cluster (3 clusters):
               Q001  Q002  Q003  Q004   Q006  Q024  Q025  TP_ESCOLA  \
cluster_nome                                                          
Intermediário  3.00  3.35  2.08  1.84   2.14  0.35   1.0       0.14   
Privilegiado   3.71  4.06  2.39  2.19   3.43  0.68   1.0       0.41   
Vulnerável     5.11  5.31  3.46  3.37  10.44  2.12   1.0       0.92   

               TP_COR_RACA  NU_NOTA_MEDIA  
cluster_nome                               
Intermediário         0.23         478.96  
Privilegiado          0.37         601.72  
Vulnerável            0.69         675.68  

Média das Notas por Área do Conhecimento (po

<Figure size 1200x800 with 0 Axes>

In [1]:
import pandas as pd
import numpy as np
import unicodedata
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from scipy.stats import f_oneway

# Function to remove accents and standardize
def normalizar_texto(texto):
    if pd.isnull(texto):
        return texto
    texto = str(texto).strip().title()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Dictionary: capitals of the Northeast -> state acronyms
capitais_para_uf = {
    'São Luís': 'MA', 'Teresina': 'PI', 'Fortaleza': 'CE', 'Natal': 'RN',
    'João Pessoa': 'PB', 'Recife': 'PE', 'Maceió': 'AL', 'Aracaju': 'SE', 'Salvador': 'BA'
}
capitais_para_uf_normalizado = {normalizar_texto(k): v for k, v in capitais_para_uf.items()}

# Load ENEM data (adjust path if needed, assuming it's accessible or a placeholder for the VM)
# IMPORTANT: The user provided a local path C:\Users\CWS\Documents\TCC\Meu tcc\microdados_enem_tratados2.csv
# This path is not accessible to the Python interpreter in the virtual environment.
# I will assume the file is in the current working directory for demonstration, or will need to ask the user to upload it.
# For now, I'll use a placeholder for `caminho_arquivo` and let the user know if the file isn't found.
# If the file needs to be uploaded, the user will get an error about file not found, which will prompt them to upload.
caminho_arquivo = r"C:\Users\CWS\Documents\TCC\Meu tcc\microdados_enem_tratados2.csv"

try:
    df = pd.read_csv(caminho_arquivo, sep=";", encoding="utf-8")
except FileNotFoundError:
    print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado. Por favor, certifique-se de que o arquivo está no diretório correto ou forneça o caminho completo e acessível.")
    exit() # Exit the script if file not found

df['NO_MUNICIPIO_PROVA_NORMALIZADO'] = df['NO_MUNICIPIO_PROVA'].apply(normalizar_texto)
df['SG_UF_ESC'] = df['NO_MUNICIPIO_PROVA_NORMALIZADO'].map(capitais_para_uf_normalizado)

ufs_nordeste = ['MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA']
df_nordeste = df[df['SG_UF_ESC'].isin(ufs_nordeste)].copy()

df_nordeste['NU_NOTA_MEDIA'] = df_nordeste[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']].mean(axis=1)

variaveis = [
    'Q001', 'Q002', 'Q003', 'Q004', 'Q006', 'Q024', 'Q025',
    'TP_ESCOLA', 'TP_COR_RACA', 'NU_NOTA_MEDIA'
]
df_cluster = df_nordeste[variaveis].dropna().copy()

# Encode categorical variables
encoder = LabelEncoder()
df_cluster['TP_ESCOLA'] = encoder.fit_transform(df_cluster['TP_ESCOLA'])
df_cluster['TP_COR_RACA'] = encoder.fit_transform(df_cluster['TP_COR_RACA'])

# Select numerical columns for clustering
X = df_cluster.select_dtypes(include=[np.number]).values

# Remove outliers using Z-score
from scipy.stats import zscore
limite_zscore = 3
z_scores = np.abs(zscore(X))
filtro_sem_outliers = (z_scores < limite_zscore).all(axis=1)
X_sem_outliers = X[filtro_sem_outliers]
df_cluster_sem_outliers = df_cluster[filtro_sem_outliers].copy()

# Normalize the data after outlier removal
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sem_outliers) # Renamed to X_scaled to avoid confusion with original X

# Define number of clusters
n_clusters = 3

# Apply K-means to the scaled data
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # Added n_init to suppress warning
clusters = kmeans.fit_predict(X_scaled)

# Add cluster information to the filtered DataFrame
df_cluster_sem_outliers['cluster'] = clusters
df_cluster_sem_outliers['SG_UF_ESC'] = df_nordeste.loc[df_cluster_sem_outliers.index, 'SG_UF_ESC']

# Rename clusters for interpretability
cluster_names = {
    0: 'Intermediário',
    1: 'Vulnerável',
    2: 'Privilegiado'
}
df_cluster_sem_outliers['cluster_nome'] = df_cluster_sem_outliers['cluster'].map(cluster_names)

# Convert to ordered category
df_cluster_sem_outliers['cluster'] = pd.Categorical(
    df_cluster_sem_outliers['cluster_nome'],
    categories=['Vulnerável', 'Intermediário', 'Privilegiado'],
    ordered=True
)

# --- Apply ANOVA ---
# Select numerical columns for ANOVA (excluding 'cluster' and 'cluster_nome' for direct stats)
# We want to test the original variables against the cluster assignments
numerical_cols_for_anova = [col for col in variaveis if col in df_cluster_sem_outliers.columns] # Ensure they are in the final df

print("\n--- Resultados da ANOVA para cada variável por Cluster ---")
print("Hipótese Nula (H0): Não há diferença significativa nas médias da variável entre os clusters.")
print("Hipótese Alternativa (H1): Há uma diferença significativa nas médias da variável entre pelo menos dois clusters.")
print("Um p-valor menor que 0.05 (nível de significância comum) sugere rejeitar a H0.")

for col in numerical_cols_for_anova:
    # Get data for each cluster
    groups = [df_cluster_sem_outliers[df_cluster_sem_outliers['cluster_nome'] == c][col].dropna()
              for c in df_cluster_sem_outliers['cluster_nome'].unique()]

    # Ensure all groups have at least one observation
    valid_groups = [g for g in groups if len(g) > 0]

    if len(valid_groups) < 2:
        print(f"\nVariável: {col} - Não é possível realizar ANOVA, menos de 2 grupos com dados.")
        continue

    # Perform one-way ANOVA
    f_statistic, p_value = f_oneway(*valid_groups)

    print(f"\nVariável: {col}")
    print(f"  F-estatística: {f_statistic:.4f}")
    print(f"  P-valor: {p_value:.4f}")

    if p_value < 0.05:
        print("  Conclusão: Rejeitar H0. Há diferença estatisticamente significativa entre as médias dos clusters.")
    else:
        print("  Conclusão: Não rejeitar H0. Não há diferença estatisticamente significativa entre as médias dos clusters.")


--- Resultados da ANOVA para cada variável por Cluster ---
Hipótese Nula (H0): Não há diferença significativa nas médias da variável entre os clusters.
Hipótese Alternativa (H1): Há uma diferença significativa nas médias da variável entre pelo menos dois clusters.
Um p-valor menor que 0.05 (nível de significância comum) sugere rejeitar a H0.

Variável: Q001
  F-estatística: 19090.4825
  P-valor: 0.0000
  Conclusão: Rejeitar H0. Há diferença estatisticamente significativa entre as médias dos clusters.

Variável: Q002
  F-estatística: 20001.2276
  P-valor: 0.0000
  Conclusão: Rejeitar H0. Há diferença estatisticamente significativa entre as médias dos clusters.

Variável: Q003
  F-estatística: 12094.1702
  P-valor: 0.0000
  Conclusão: Rejeitar H0. Há diferença estatisticamente significativa entre as médias dos clusters.

Variável: Q004
  F-estatística: 14788.7529
  P-valor: 0.0000
  Conclusão: Rejeitar H0. Há diferença estatisticamente significativa entre as médias dos clusters.

Variáv

C:\Users\CWS\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)
